In [1]:
import pandas as pd
from pandas.tseries.offsets import Hour, Minute, Second # ...
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import itertools as it
import pytz
import numpy as np
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster

from sklearn.manifold import TSNE
from sklearn.manifold import LocallyLinearEmbedding

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering #kernel
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import FeatureAgglomeration
from sklearn.cluster import AffinityPropagation

from yellowbrick.cluster import KElbowVisualizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

KeyboardInterrupt: 

# DataSet概要
## Data fields

- 日付 -犯罪事件のタイムスタンプ
- カテゴリ -犯罪事件のカテゴリ（train.csvのみ）。これは、予測するターゲット変数です。
- 説明 -犯罪事件の詳細な説明（train.csvのみ）
- DayOfWeek -曜日
- PdDistrict-警察署地区の名前
- 解決-犯罪事件がどのように解決されたか（train.csvのみ）
- 住所-犯罪事件のおおよその 住所
- X-経度
- Y-緯度

# First EDA  [Exploratory Data Analysis]
- データを俯瞰的に見る。csvの内部状況の確認(columns,null,mean,std)
- 以下から簡単にわかる事
    1. タイムスタンプが存在し、それがカテゴリ化されてもいる。
    2. タイムスタンプに対して曜日がカテゴライズされている。
    3. タイムスタンプは犯罪発生日であるのでロケーションがカテゴライズされている。
    4. ロケーションに対してX,Yは緯度経度であると推測できる。(これは推測せずともデータセットの説明に記述がある、しかしこれが何かを推測する事も大事。推測の過程で新たなFEを発見する可能性がある)
    5. X,YはColumnsとして分かりにくいという場合にはLongitude, latitudeに名前を変えるという事も行う。(Lon,Latでも可)
        - ライブラリによっては、緯度経度を渡す位置が逆になる事もあるのでライブラリを確認する。
- 可能であると思われる可視化
    1. 通常の可視化(Visualization)プロット
    2. 地図を可視化し、犯罪がどの地区でよく発生しているのかを可視化できる
    3. 何故その地区で犯罪発生件数が多いのか、SanFranciscoの別のデータ(店舗、居住区画、居住区画住宅価格、種族別居住有無し)と併用して様々な見解を得る事が出来る。
    4. 犯罪発生率とGDP, 進学率, 過去のSingle世帯の多さ, 産業の発生・発展におけるタイムスタンプ的犯罪発生率の増減, サンフランシスコ全体の物価指数の伸び, 都市開発のタイムスタンプ, サンフランシスコの居住者の増減,　等のデータセットと組み合わせを行い、様々な知見が得られる。これは犯罪は増減する理由を根本的に説明する起源現象の解明に当たるであろうから、これそのものが該当する他の州の基礎コホートとしても成立する可能性がある(その場合、産業の発生発展は除外するべきであると推理する、これは州によって変化するからである)。
- 可能であると思われる変換
    1. 階層型カテゴライズ
    2. クラスター変換作成、階層型クラスターの作成
    3. 分布が正規分布に準じていない場合Box-Cox変換を使用した正規分布への変換
    4. (変換後or無変換)正規分布を使用したガウス混合クラスタリング解析
    5. PCAを使用して分散を維持したまま次元削減を行い、クラスタリングを可視化する。
        - 可視化はあくまで知見を得る為に使用する、知見を得る事によってFeatureEnginieringの精度を上げる等の方法論的解釈を可視化によって行う。
- 別Notebookでの論理哲学的事象発生探究の項から抜粋
    - ある事象の発生はその事象が帰結した結果の付随する事象の総和である。これは何故その事象が発生するのか、というのを論理的に解説する物である。

In [ ]:
train = pd.read_csv('D:/JupyterNotebook/SanFranciscoCrimeDataset/train.csv')
test = pd.read_csv('D:/JupyterNotebook/SanFranciscoCrimeDataset/test.csv')

In [ ]:
train.shape, test.shape

## train

In [ ]:
train.head(3).style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
train.describe().style.background_gradient(cmap='mako_r', text_color_threshold=0.02) # この記述統計量はあまり意味が無い、XとYは経度緯度を表している。testも同様

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

## test

In [ ]:
test.head(3).style.background_gradient(cmap='rocket', text_color_threshold=0.02)

In [ ]:
test.describe().style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

-----------
-----------

## unique

In [ ]:
train.head(3).style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
train['PdDistrict'].unique()

In [ ]:
train['Resolution'].unique()

In [ ]:
train['Category'].unique() # この特徴量が予測子。

In [ ]:
train['DayOfWeek'].unique()

In [ ]:
train['Address'].unique(), len(train['Address'].unique())

In [ ]:
len(train['Descript'].unique())

In [ ]:
train.iloc[1]

--------
# Visualization
1. DayOfWeekに対してCategoryの可視化、どの週が最も犯罪発生率が高いか。
2. CategoryのHistgram
3. PdDistrictのHistgram,つまりどの署が最も対応した数が多いか。
4. タイムスタンプに対しての犯罪発生率の増減、季節性の視認。
5. タイムスタンプの年毎の犯罪増減率と、その関係性を示す人流、GDP等の社会動態経済動態の知見。

In [ ]:
def visualization_crime(value, tight=False):
    with plt.style.context('fivethirtyeight'):
        fig, ax = plt.subplots(1, 1, figsize=(19, 6))
        sns.histplot(x=value, data=train, kde=True, palette='rocket', ax=ax)
        ax.tick_params(axis='x', rotation=90)
        if tight == True:
            fig.tight_layout()

### Category Visualization

In [ ]:
visualization_crime('Category', tight=False)

### DayOfWeek Visualization

In [ ]:
visualization_crime('DayOfWeek')

### PdDistrict Visualization

In [ ]:
visualization_crime('PdDistrict')

### Resolution Visualization

In [ ]:
visualization_crime('Resolution')

### Latitude Visualization

In [ ]:
visualization_crime('X')

------

## Seaborn ColorPalette
[color_palette_Library](https://seaborn.pydata.org/generated/seaborn.color_palette.html#seaborn.color_palette)

-------

- 緯度経度情報が存在するので散布図として可視化する。
    - こうする事によって散布図の形が街の形を表示する。これは件数が増えれば増える程視覚的に明確になる。
    - ここでわかる事は、データのLatitude, Longitudeが座標を明確に示しており、正確であるという事が分かる。可視化せずにLatitude, Longitudeを使用するのは経験上危険である。
    - 何処でどのようなCrimeが発生しているのかが一目瞭然になる。以下の散布図は250000/ALLを元に表示している、これはデータが実際の緯度経度であるのかを明確にするために行っている。
        - 条件指定を行って特定のCrimeのみを表示させる事も可能。これは下記のFolium参照

In [ ]:
# 街の形が視覚的に見えてくる。
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize=(19, 19))
    sns.scatterplot(data=train.iloc[:250000], x='X', y='Y', alpha=0.6, palette='rocket', hue='Category', size='Category') # 878049 
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')

- DayOfWeekによって細分化したデータ。データ数が多いので少しわかりにくいが、週末にかけて左寄りの形状を持つ。
- これらの問題は全てfoliumで解決する。これはあくまでも実装前テスト表示に過ぎない。

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize=(19, 19))
    sns.scatterplot(data=train.iloc[:50000], x='X', y='Y', alpha=0.6, palette='flare', hue='DayOfWeek',
                    size='DayOfWeek', sizes=(20, 200), markers=True) # 878049 

----
# 1. Folium 
>2.では年間データの推移、クラスター化によるAnimation表示をFoliumで実装している。
[Foliumライブラリ](https://python-visualization.github.io/folium/)

- 関数化
- pandas queryを使用して高速化をはかりつつ(実際にはNumexpr)複数の条件によってデータのHeatmap表示を行えるようにしている。
    - queryによって取得したASSAULTはダウンタウン、チャイナタウン周辺に集中している事がわかる。ベイエリアでも比較的多いのには集団が集まりやすい傾向にあるからであると推測できる。
    - 集団がつまればCrimeに発展する確率が上がるのは以下のEDAで明確になるので以下参照。

In [ ]:
def query_heat_map(query, location=[37.774599, -122.425892]):
    train_query = train.query(query).loc[:, ['Y', 'X']]
    if train_query.shape[0] == 0:
        print('Either the query is failing or there is no data itself.')
    m = folium.Map(location=location, zoom_start=13, tiles='CartoDB dark_matter') # HeatmapPlot-best-tiles : cartodbdark_matter
    train_query_geo_list = train_query.values.tolist()
    HeatMap(train_query_geo_list, blur=2, radius=3).add_to(m)
    #m.save('SanFrancisco-Crime-geo.html') # add to variable string
    return m

In [ ]:
query_heat_map("Category=='ASSAULT' & Resolution=='ARREST, BOOKED'");

---------
---------
# タイムスタンプ操作
1. indexをDates使用して読み込む場合
2. DatesをTimestampとして読み込みを行い、Timestampのデータを分割変換する。分秒単位までは必要はあまりないと思われるが、時単位まで単体化する事によって有効なlineplotを行えると推測できる。これによって、効果的な第三者への視覚化が行える。例：.count()の統計量を利用して何時が最も犯罪率が多いか、等。
3. 時系列データが存在する場合、積極的に時系列に変換して使用するのがベストプラクティス

# 1. indexをTimestampとして読み込む。
- どのような変換が考えられるか。
- 実際にこのデータをPre-Processingとして使用する訳ではない。

In [ ]:
train_timestamp = pd.read_csv('D:/JupyterNotebook/SanFranciscoCrimeDataset/train.csv', parse_dates=True, index_col='Dates')

In [ ]:
train_timestamp.loc['2015-05-13'].head(2)

In [ ]:
train_timestamp.index

- 文字列は統計処理されない。これは緯度経度の各平均になっており、この平均をFoliumの基礎map-locationに使用する事も出来る。

In [ ]:
train_timestamp.loc['2015'].mean()

- 2015年のデータは途中までの期間しかないのか？それとも犯罪が減少する要因がアメリカ、特にサンフランシスコであったのか？という事を推理する。
    - これは実際にはデータが特定期間までしかない事が減少要因、max(),min()等で確認できる。

In [ ]:
train_timestamp.loc['2015'].count()

In [ ]:
train_timestamp.loc['2013'].count()

In [ ]:
mc = train_timestamp.groupby(level=0)

In [ ]:
mc.count().head(3)

# 2. DatesはColumnsのままtimestampとして読み込む。

In [ ]:
train_plot = pd.read_csv('D:/JupyterNotebook/SanFranciscoCrimeDataset/train.csv', parse_dates=True)

- info(),describe()は上記で実行済みなのでこのlineでは実行しない。
- 以下では重複している行がどれだけ存在しているかを可視化、削除

In [ ]:
train_plot.duplicated().sum()

----
----
# シグマクリップ SigmaClip
- シグマクリップを使用して外れ値の除去を行う。間違っている日付等(32日等)を一括で削除可能な方法。
- 最後の行はサンプル平均のロバスト推定を使用している、0.74はガウス分布の四分位範囲から得られる値。ここでは実際の構文は記述しない。
- queryを使用して高速化を行う。

In [ ]:
frequency_c = pd.DataFrame(train_plot['Category'].value_counts(normalize=True))
frequency_c.reset_index(inplace=True)
frequency_c.rename({'index': 'Category', 'Category': 'Freq'}, axis=1, inplace=True)
frequency_c['Cumsum_Freq'] = frequency_c['Freq'].cumsum()

In [ ]:
frequency_c = frequency_c.set_index('Category')

In [ ]:
freq_quantiles = np.percentile(frequency_c['Cumsum_Freq'], [25, 50, 75]) # array([0.84974358, 0.96996523, 0.99674847])

In [ ]:
qt = freq_quantiles[1] # 0.9699652297309147

In [ ]:
sigma = 0.74 * (freq_quantiles[2] - freq_quantiles[0]) # 0.10878362141520569

In [ ]:
query_freq = frequency_c.query("(Cumsum_Freq > @qt - 5 * @sigma) & (Cumsum_Freq < @qt + 5 * @sigma)") # Cumsum_Freq is Data_Columns

In [ ]:
query_freq.reset_index(inplace=True)

In [ ]:
query_freq_list = list(query_freq['Category'])

In [ ]:
query_freq_list

In [ ]:
train_sigma = train.loc[train['Category'].isin(query_freq_list)]

In [ ]:
train_sigma.head()

In [ ]:
train_sigma['Dates'] = pd.to_datetime(train_sigma['Dates']) # Not Copy is SettingCopyWarning

In [ ]:
test['Dates'] = pd.to_datetime(test['Dates'])

--------
--------
# Second EDA
- 上記で得られた知見や、データ処理の結果を使用してより細分化したEDAを実行し、知見を得、第三者へのプレゼンテーションEDA資料の１つとする。
- 各可視化の説明詳細はある程度省く。

# どのCrimeが最も多いか？

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize=(19, 19))
    sns.countplot(y='Category', data=train_sigma, order=train_sigma['Category'].value_counts().index,  palette='rocket')
    ax.tick_params(axis='x', rotation=45)
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha="right")

- 警察署別によってどの事件への対応が最も多いか？
    - これは地図上に警察署をロケーションポイントとして表示させる事によってより明確に出来る。
    - 余談：MISSION署は非常に危険なようだ、、。DRUGに関してはTENDERLOINが最も多い、これは有名な話でテンダーロインには観光に行くなというのは定説。他国を批判する気はないのだが、テンダーロインはサンフランシスコの中でもかなり治安が悪い事で有名。ASSAULTも同様に多い。

In [ ]:
cross_tab_train = pd.crosstab(train_sigma['Category'], train_sigma['PdDistrict'])

In [ ]:
cross_tab_train.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
stack_sigma_list = cross_tab_train.stack().reset_index().rename(columns= {0:'value'})

- 数値的可視化からバープロットによる視覚的可視化を行う。両方行う事も重要、知見の多様化を行える。

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize= (15,10))
    sns.barplot(x=stack_sigma_list['PdDistrict'], y=stack_sigma_list['value'], hue=stack_sigma_list['Category'],  palette='rocket')
    ax.set_title('Categories Count per District')
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')

----
----
- 0.8ベースで表示した場合以下

In [ ]:
frequency_c = pd.DataFrame(train_plot['Category'].value_counts(normalize=True))
frequency_c.reset_index(inplace=True)
frequency_c.rename({'index': 'Category', 'Category': 'Freq'}, axis=1, inplace=True)
frequency_c['Cumsum_Freq'] = frequency_c['Freq'].cumsum()
frequency_50_list = list(frequency_c.loc[frequency_c['Cumsum_Freq'] < 0.8, 'Category'])

In [ ]:
train_50_list = train.loc[train['Category'].isin(frequency_50_list)]

In [ ]:
train_50_list['Dates'] = pd.to_datetime(train_50_list['Dates'])

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize=(19, 19))
    sns.countplot(y='Category', data=train_50_list, order=train_50_list['Category'].value_counts().index,  palette='rocket')
    ax.tick_params(axis='x', rotation=45)

In [ ]:
ct_50_list = pd.crosstab(train_50_list['Category'], train_50_list['PdDistrict'])
ct_50_list.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
stack_50_list = ct_50_list.stack().reset_index().rename(columns= {0:'value'})

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize= (15,10))
    sns.barplot(x=stack_50_list['PdDistrict'], y=stack_50_list['value'], hue=stack_50_list['Category'], palette='rocket')
    ax.set_title('Categories Count per District')

---------
---------
# Pre-Processing & Feature Enginiering
- 上記で得られた知見を利用してデータの変換と特徴量エンジニアリングを行う。
- 日時変換には国土交通省指定の時間分割表を参照している。
- 解説、に該当する特徴量に関しては、これ以降のPipelineで削除する。これは変換は非常に難しい事と（NLPを利用等）testセットにこの特徴量が存在していないので、解説をテストセットに対して追加するのはベストプラクティスとはなりえないであろうという事。ただし、正確な予測をする場合、完全削除する事は悪手でしかない。

In [ ]:
train_sigma.shape, train_50_list.shape

In [ ]:
def time_group(date):
    
    date['Date'] = date['Dates'].dt.date
    date['Year'] = date['Dates'].dt.year
    date['Month'] = date['Dates'].dt.month
    date["Day"] = date["Dates"].dt.day
    date["Hour"] = date["Dates"].dt.hour
    date["Minute"] = date["Dates"].dt.minute
    date["Second"] = date["Dates"].dt.second
    
    ca = calendar()
    holidays = ca.holidays(start=date['Dates'].min(), end=date['Dates'].max())
    date['Holiday']= date['Dates'].dt.date.astype('datetime64').isin(holidays)
    # Week_replace
    week_mapping = {'Saturday': 5, 'Sunday': 6, 'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4}
    date['Week_Mapping'] = date['DayOfWeek'].map(week_mapping)
    
    # https://www.jma.go.jp/jma/kishou/know/yougo_hp/saibun.html 国土交通省：気象庁[定義区分] time scale
    def hour_Segmentation(x):
        if x >= 3 and x < 9:
            return 0
        elif x >= 9 and x < 15:
            return 1
        else:
            return 2
        
    def morning_Or_Afternoon(x): # np.where
        if x >= 0 and x > 12:
            return 0
        else:
            return 1
        
    def daytime_Or_Nighttime(x): # np.where
        if x >= 9 and x > 18:
            return 0
        else:
            return 1
    
    def default_Work_Time_Hours(x): # np.where
        return 8 <= x <= 18
    
    def weekday_and_ends(x):
        return np.where(x < 5, 'Weekday', 'Weekend')
    
    # Q1:1 , Q2:2, Q3:3, Q4:4
    def quarter(x):
        if x >= 1 and x <= 3:
            return 1
        elif x > 3 and x <= 6:
            return 2
        elif x > 6 and x <= 9:
            return 3
        elif x > 9 and x <= 12:
            return 4
    
    # T1:1, T2:2, T3:3
    def month_sep(x):
        if x >= 1 and x < 10:
            return 1
        elif x >= 10 and x < 20:
            return 2
        elif x >= 20 and x <= 31:
            return 3
    
    def street_type(x):
        street_list = x.split(' ')
        for index in range(len(street_list)):
            fo_list = street_list[index]
            if len(fo_list) == 2 and fo_list not in ['OF', 'US', 'LA', 'of']:
                return fo_list
        
    date['Street_type'] = date['Address'].apply(street_type)
   
    date['HourGroup'] = date["Hour"].apply(hour_Segmentation)
    date['MAGroup'] = date['Hour'].apply(morning_Or_Afternoon)
    date['DNGroup'] = date['Hour'].apply(daytime_Or_Nighttime)
    date['DworkGroup'] = date['Hour'].apply(default_Work_Time_Hours)
    date['Week_cat'] = date['Week_Mapping'].apply(weekday_and_ends)
    date['Month_quarter'] = date['Month'].apply(quarter)
    date['Month_sep'] = date['Day'].apply(month_sep)
    
    return date

In [ ]:
train_sigma_1 = train_sigma.copy()

In [ ]:
train_50_list_1 = train_sigma.copy()

In [ ]:
train_group = time_group(train_sigma_1) # Main_train_group

In [ ]:
test_group = time_group(test) # Main_Test

In [ ]:
train_group.shape, test_group.shape

In [ ]:
train_group_1 = time_group(train_50_list_1)

------
### 変換後のデータを確認する。
- 明らかに2015年のデータが少ない、これは上記でも記述した通りデータが途中期間までしか存在していない為

In [ ]:
year_data = pd.DataFrame(train_group.groupby('Year')['Category'].count())
year_data_1 = pd.DataFrame(train_group_1.groupby('Year')['Category'].count())

In [ ]:
year_data.reset_index(inplace=True)

In [ ]:
year_data.T.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
year_data.drop(12, axis=0, inplace=True)

In [ ]:
year_data.T.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

--------
--------
# 新特徴量による可視化

In [ ]:
hour_vs_cate = train_group.groupby(['Category', 'HourGroup'], as_index=False).count()

In [ ]:
hour_vs_cate.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
hour_vs_cate_pv = hour_vs_cate.pivot(index='HourGroup', columns='Category', values='Dates') #.fillna(0)

In [ ]:
hour_vs_cate_pv.style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
fig, ax = plt.subplots(figsize=(50, 5)) 
sns.heatmap(hour_vs_cate_pv.apply(lambda x:x/sum(x),axis=0), square=True, annot=True)

In [ ]:
with plt.style.context('fivethirtyeight'):
    de_time = train_group.groupby([train_group.Week_cat, train_group.Hour])['Category'].count()
    de_time.plot(figsize=(19, 6))

- Weekend,dayでのCrime発生数
- 12時にCrimeが発生する確率が上昇する。これは集団を形成する為に、そのような状況が必然的に発生しうる状況が作成されるという事でもある。
- 2番目の可視化でも12時にCrime発生が一時的に上昇する。

In [ ]:
with plt.style.context('fivethirtyeight'):
    de_time = train_group.groupby([train_group.Week_cat, train_group.Hour])['Category'].count()
    de_time.loc['Weekday'].plot(figsize=(19, 6), label='Week_Day')
    de_time.loc['Weekend'].plot(figsize=(19, 6), label='Week_End')
    plt.title('Week, Day or End Crimes')
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')

In [ ]:
with plt.style.context('fivethirtyeight'):
    pt = pd.pivot_table(train_group.loc[:, ['Hour', 'Category']], index="Hour", columns="Category", aggfunc=len, fill_value=0)
    pt.plot(figsize=(30,10))
    plt.title('Hour Crimes')
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
    #plt.tight_layout()

In [ ]:
with plt.style.context('fivethirtyeight'):
    pt = pd.pivot_table(train_group.loc[:, ['Year', 'Category']], index='Year', columns='Category', aggfunc=len, fill_value=0)
    pt.plot(figsize=(30,10))
    #plt.gca().set_xticklabels(['2010', '2011', '2012', '2013', '2014', '2015'])
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(figsize=(30, 10))
    ax = sns.lineplot(x=year_data['Year'], y=year_data['Category'])
    ax.set_title('Year Crimes')

In [ ]:
with plt.style.context('fivethirtyeight'):
    pt = pd.pivot_table(train_group.loc[:, ['Month', 'Category']], index='Month', columns='Category', aggfunc=len, fill_value=0)
    pt.plot(figsize=(30,10))
    plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')

In [ ]:
test_1 = train_group.loc[:, ['Year', 'Category']].reset_index().merge(train_group.loc[:, ['Y', 'X']].reset_index(), on=['index']).set_index('Year')

In [ ]:
test_g = train_group.groupby(["Category", "Year"]).count()

In [ ]:
test_g.unstack()['Dates'].style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

-----
-----
# 2. Folium
- Foliumによる動的な可視化を行う。

In [ ]:
time_series = list(np.sort(train_group['Year']))

In [ ]:
train_query_move = train_group.query("Category=='ASSAULT'").loc[:, ['Y', 'X', 'Year']]

In [ ]:
train_query_move = train_query_move.set_index('Year', drop=False)

In [ ]:
test_11 = np.sort(train_query_move.index.unique())

In [ ]:
train_move_list = [train_query_move.loc[i].values.tolist() for i in np.sort(train_query_move.index.unique())]

In [ ]:
train_query = train.query("Category=='ASSAULT' & Resolution=='ARREST, BOOKED'").loc[: ,['Y','X']]

In [ ]:
train_query_geo_list = train_query.values.tolist() # default feat map array

In [ ]:
index_time = np.sort(train_query_move.index.unique()).tolist()

In [ ]:
# データのソートとindexの追加、そしてヒートマップサイズの調整

In [ ]:
# 使用するかわからないが一応作成したロケーションのリスト基本はdict型を使用するのが効率的にもよい。
'''
down_town = [[37.77493, -122.419416]]
union_square = [[37.786163522, -122.404498382]]
fishermans_wharf = [[37.80499678, -122.409331696]]
china_town = [[37.790163506, -122.404331716]]
soma = [[37.777311, -122.411083]]
oak_street = [[37.77412, -122.431384]]
twin_peaks = [[37.751586275, -122.447721511]]
southeast = [[37.7369444, -122.3941667]]
bernal_heights = [[37.744385, -122.417046]]
mission_district = [[37.76, -122.42]]
noe_valley = [[37.7502, -122.4337]]
haight_ashbury = [[37.770015, -122.446937]]
presidio = [[37.798085, -122.466538]]
'''

sanfrancisco_location = {'down_town': [[37.77493, -122.419416]], 'union_square': [[37.786163522, -122.404498382]],
                         'fishermans_wharf': [[37.80499678, -122.409331696]], 'china_town': [[37.790163506, -122.404331716]],
                         'soma': [[37.777311, -122.411083]], 'oak_street': [[37.77412, -122.431384]],
                         'twin_peaks': [[37.751586275, -122.447721511]], 'southeast': [[37.7369444, -122.3941667]],
                         'bernal_heights': [[37.744385, -122.417046]], 'mission_district': [[37.76, -122.42]],
                         'noe_valley': [[37.7502, -122.4337]], 'height_ashbury': [[37.770015, -122.446937]], 'presidio': [[37.798085, -122.466538]]}

In [ ]:
def geo_polygon(lat_lon):
    '''lat_lonを基準としてgeographic quarter(NW, NE, SW, SE)を求める'''
    sw, nw, se, ne = [(lat + py * pow(10, -3), lon + px * pow(10, -3)) for px, py in it.product([-1, 1], [-1, 1]) for lat, lon in lat_lon]
    return [sw, se, ne, nw]

In [ ]:
def location_point(location):
    '''LocationPointを一括で表示させる関数：辞書による変数'''
    for location_name, lat_lon in location.items():
        lat_lon_locate = geo_polygon(lat_lon)
        folium.Polygon(locations=lat_lon_locate, color="red", weight=1, fill=True, fill_opacity=0.1, popup=location_name).add_to(m)

def location_point_1(location):
    for location_name, lat_lon in location.items():
        lat_lon_locate = geo_polygon(lat_lon)
        folium.Polygon(locations=lat_lon_locate, color="red", weight=1, fill=True, fill_opacity=0.1, popup=location_name).add_to(m_1)

In [ ]:
m = folium.Map(location=[37.774599, -122.425892], zoom_start=13, tiles='CartoDB dark_matter')

In [ ]:
location_point(sanfrancisco_location)

In [ ]:
#HeatMapWithTime(train_query_geo_list,auto_play=False,radius=40,max_opacity=1,gradient={0.1: 'blue', 0.25: 'lime', 0.5:'yellow',0.75: 'red'}).add_to(m)
HeatMapWithTime(train_move_list, index=index_time, auto_play=False, radius=1 , max_opacity=1, gradient={0.1: 'blue', 0.25: 'lime', 0.5:'yellow',0.75: 'red'}).add_to(m)

## 2003-2015年までのASSAULTの散布図
- 再生ボタンで動的に表示されます。fpsはデフォルトの10を使用しているが、スライドバーを上げる事によって上げられます。
- 赤枠はロケーションポイントで、クリックするとロケーション名が表示されます。赤枠にしている理由はこのような表示が可能という事を見せているだけです。本来ピンが最も効果的であると思われます。

In [ ]:
m

In [ ]:
m_1 = folium.Map(location=[37.774599, -122.425892], zoom_start=13, tiles='CartoDB dark_matter')

In [ ]:
marker_cluster = MarkerCluster().add_to(m_1)

In [ ]:
location_point_1(sanfrancisco_location)

In [ ]:
for point in range(0, len(train_query_geo_list)):
    folium.Marker(train_query_geo_list[point], popup='A').add_to(marker_cluster)

### 2003-2015年までのASSULTのクラスタリング表示
- マップをスクロールする事によっても、対象の区画をクリックする事でもクラスタを可視化する事が出来ます。緯度経度が正確に記録されている為、最小単位までスクロールダウンした時、どの位置で事件が起きているのか明確にピンで見る事が出来ます。作成したPCの性能上の問題で、ピンには全て'A'と表示されていますが、事件の内容そのものを表示させる事も可能、最小単位のマーカーにはそれが何であるのかを表示させられるpopupが存在するので、そこにCrimeの詳細を表示させられる。
- これにはlat_lonの管理にIDを連結させ、その連結項目からCrimeの詳細を取り出してpopupで表示。

In [ ]:
m_1

----
----
## 各Crimeが週のうち何時起きているのかを可視化する。
- これによって事件の性質や、その事件の背景がより明確に知見としての推理を得られる。

In [ ]:
sorted_map = {'Saturday': 0, 'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6}

In [ ]:
def dayOfWeek_plot(data, string='ASSAULT'):
    with plt.style.context('fivethirtyeight'):
        week_data = pd.DataFrame(data[data['Category'] == string].groupby(by=['DayOfWeek'])['Category'].count()).reset_index()
        week_data['SortedDayOfWeek'] = week_data['DayOfWeek'].map(sorted_map)
        week_data = week_data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

        fig, ax = plt.subplots(1, 1, figsize = (19, 6))
        ax = sns.lineplot(x=week_data['DayOfWeek'], y=week_data['Category'])
        ax.set_title('{} Crimes. Week'.format(string))

In [ ]:
dayOfWeek_plot(train_group, 'ASSAULT')

In [ ]:
list(train_group['Category'].unique()); # test code

In [ ]:
# add def dayOfWeel_plot
for string in list(train_group['Category'].unique()):
    dayOfWeek_plot(train_group, string)

In [ ]:
with plt.style.context('fivethirtyeight'):
    plt.figure(figsize=(19, 6))
    sns.kdeplot(train_group.groupby('Date').count().iloc[:, 1], shade=True, palette=['#682F2F'])
    plt.xlabel('Incidents')
    plt.ylabel('Density')

### Holiday 

In [ ]:
ct_holiday_cat = pd.crosstab(train_group['Category'], train_group['Holiday'])
ct_holiday_cat.T

In [ ]:
stacked = ct_holiday_cat.stack().reset_index().rename(columns={0:'value'})

In [ ]:
stacked.loc[stacked['Holiday'] == False, 'value'] /= train_group.loc[train_group['Holiday'] == False, 'Holiday'].count()
stacked.loc[stacked['Holiday'] == True, 'value'] /= train_group.loc[train_group['Holiday'] == True, 'Holiday'].count()

In [ ]:
stacked.T

- Holiday　Plot

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize = (19, 6))
    bar = sns.barplot(x=stacked['Category'], y=stacked['value'], hue=stacked['Holiday'], palette=['#682F2F', '#F3AB60'])
    bar.set_title('Proportions of crimes during regular days vs holidays')
    ax.tick_params(axis='x', rotation=90)

In [ ]:
ct_business_hrs_cat = pd.crosstab(train_group['Category'], train_group['DworkGroup'])
ct_business_hrs_cat.T

In [ ]:
stacked = ct_business_hrs_cat.stack().reset_index().rename(columns={0:'value'})
stacked.loc[stacked['DworkGroup'] == False, 'value'] /= train_group.loc[train_group['DworkGroup'] == False, 'DworkGroup'].count()
stacked.loc[stacked['DworkGroup'] == True, 'value'] /= train_group.loc[train_group['DworkGroup'] == True, 'DworkGroup'].count()
stacked.T

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(1, 1, figsize = (19, 6))
    bar = sns.barplot(x=stacked['Category'], y=stacked['value'], hue=stacked['DworkGroup'], palette=['#682F2F', '#F3AB60'])
    bar.set_title('Proportions of crimes during regular days vs holidays')
    ax.tick_params(axis='x', rotation=90)

----------
# TimeStamp の変換
- 時系列データを営業日や四半期などの表示に切り替えてデータを推理する事も重要だが、これは変換のみにしておき省く。

In [ ]:
train_group.head(2)

In [ ]:
train_group_2 = train_group.copy()

In [ ]:
train_group_2.set_index('Dates', inplace=True)

In [ ]:
train_group_2.loc['2014'].head(2)

In [ ]:
freq_d = pd.to_datetime(train_group_2.index)

In [ ]:
train_group_freq_d = pd.DataFrame(train_group_2, index=freq_d)

In [ ]:
train_group_2.to_period('Q-DEC').head(2)

------
------
## Main-Pre-Process-Pipeline

- 次の処理で必要なのはカテゴリ化を正確に行える処理。
    - Address, Resolusion, Descriptは一意な値が正確ではない。これらの処理を行う必要がある。
    
    - Address : 済(street_type)
        - 更にそのストリートがどの区画にあるのかという特徴の追加も可能であると思われる(以下では行っていない)
       
- Category : target
- Delete Columns : Address, Dates

- モデルに渡しているpipelineではStreetTypeを削除したPipelineを使用している、これはPC環境によってはモデルの訓練に時間がかかる為、桁数の指定を行った際にカテゴリが合わなくなる状況が発生するのを避ける為に行っている。

In [ ]:
train_group.head(2)

In [ ]:
test_group.head(2)

In [ ]:
train_group.drop(['Dates', 'Descript', 'Resolution', 'Address', 'Date'], axis=1, inplace=True)

In [ ]:
test_group.drop(['Id','Dates', 'Address', 'Date'], axis=1, inplace=True)

In [ ]:
X = train_group.drop('Category', axis=1)
y = train_group['Category']

In [ ]:
data_labels = y.to_numpy()

In [ ]:
data_num = X.drop(['DayOfWeek', 'PdDistrict', 'DworkGroup', 'Week_cat', 'Holiday', 'Street_type'], axis=1)

In [ ]:
number_attribs = list(data_num)

In [ ]:
category_attribs = ['DayOfWeek', 'PdDistrict', 'DworkGroup', 'Week_cat', 'Holiday', 'Street_type']

In [ ]:
number_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

main_pipeline = ColumnTransformer([
    ("number", number_pipeline, number_attribs),
    ("categorie", OneHotEncoder(), category_attribs),
])

In [ ]:
X.columns

In [ ]:
test_group.columns

-----
# Default Transform

- 本来のPipeline Transform
- モデルの訓練時間の都合上、このNotebookではこれを使用していない。
- Clusteringは含めずに変換している。

In [ ]:
main_x = main_pipeline.fit_transform(X)

In [ ]:
main_test = main_pipeline.transform(test_group)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(main_x, data_labels, test_size=0.25, stratify=data_labels, shuffle=True)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

# Githubの仕様上FileUploadは1file25MBが上限なので、以下のクラスタリング以降のPipelineは事前実行していません。Binderか、自環境で表示させてください。

------
------
# Clustering

In [ ]:
# Error:変数格納で改善、連続的にshowする事でこのエラーは発生しない : 変数に格納しない場合、一つのlineにshow()までを格納する。
# https://stackoverflow.com/questions/69285993/kmeans-object-has-no-attribute-k
model = KMeans(init='k-means++', n_init=10)
visualizer = KElbowVisualizer(model, k=(1, 20))

In [ ]:
visualize = visualizer.fit(main_x)

In [ ]:
visualizer.show();

In [ ]:
main_cluster_data = KMeans(n_clusters=visualizer.elbow_value_).fit_predict(main_x) # 6 or 5

In [ ]:
main_cluster_test = KMeans(n_clusters=visualizer.elbow_value_).fit_predict(main_test)

In [ ]:
X_cluster = X.copy()

In [ ]:
test_cluster = test_group.copy()

- 作成したクラスタを代入する。これを再度Transformする事になるが、Transform前にClusteringを行う方がよい。

In [ ]:
X_cluster['Cluster'] = main_cluster_data

In [ ]:
test_cluster['Cluster'] = main_cluster_test

---------------
## Second Cluster transform

- Firstは実質的な最終Transformだが、index数が多すぎて通常のPC環境では処理に時間がかかるという問題が発生する。これを解決する為に、変換する前にindexの一部を削減する。この方法はベストではないむしろ悪手であるが、モデルを動かすという点では有益。これで得られる結果は正確ではないという事を理解しておく事が重要。
- まず、変換に掛ける前のクラスターを代入したデータセットからデータの一部のみを取り出す。これは、.sample()でもよいように思う(ランダムで取り出せるので)
- 取り出し前のデータセット数は50万あるので、これを10万以下にする必要があるように思う(これは調整でどうにかする。)

In [ ]:
# Test　cv=kfold　に使用する。GridSearchCV等に使われる、他のcvにも使用可能。
#from sklearn.model_selection import StratifiedKFold
#kfold =StratifiedKFold(n_splits=5,shuffle=True,random_state=42)　

In [ ]:
X_cluster.shape, X_cluster[:100000].shape # この状態でmodelに渡す

In [ ]:
# 上記の構文でもよいが、カテゴリ化する際のErrorを避ける為に、下記では.sample()を使用.(このNotebookでは使用していない。削除で対応している。)
# 他にも、streetカテゴリが変換上全て含まれない場合が多いので、以下のモデルでは削除する。これも本来は削除してはならない特徴量で実環境では追加して変換をかける事: Street_type
X_cluster.sample(10000);

-----
- 以下は本来は必要ない。これはモデルを円滑に実行させるための構文で、本来のベストプラクティスではない。モデルを動かす為以外で使用する場合は最悪の悪手である
- Streetがカテゴリ化されるため、X_clusterのフルサンプルでなければカテゴリ数が足りないと言われる可能性があるが、これはベストプラクティスではないので何度か実行すれば全て含まれる。
- 以下ではStreetを削除している。

In [ ]:
X_cluster_test = X_cluster.drop('Street_type', axis=1)

In [ ]:
test_cluster_test = test_cluster.drop('Street_type', axis=1)

In [ ]:
data_num = X.drop(['DayOfWeek', 'PdDistrict', 'DworkGroup', 'Week_cat', 'Holiday', 'Street_type'], axis=1)
number_attribs = list(data_num)
category_attribs = ['DayOfWeek', 'PdDistrict', 'DworkGroup', 'Week_cat', 'Holiday']

number_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

main_pipeline = ColumnTransformer([
    ("number", number_pipeline, number_attribs),
    ("categorie", OneHotEncoder(), category_attribs),
])

In [ ]:
main_x_1 = main_pipeline.fit_transform(X_cluster_test.sample(50000))

In [ ]:
main_test_1 = main_pipeline.transform(test_cluster_test)

-----
# このNotebookで使用する、最終的な訓練セットと検証セット

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(main_x_1, data_labels[:50000], test_size=0.25, shuffle=True) # stratify=data_labels

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

--------
# Model

- GridSearchCV,RandomizedSearchCV,CV等はこのNotebookを作成した低スペック環境上使用していません。
- 上記の通り、Modelのハイパーパラメータの調整は行っていないので性能はかなり低い。訓練の可視化も行っていないのでどのような状態になっているのかも視覚的にはわかりません。
- Modelの訓練に関しては私の作成している別のNotebookを参照して下さい。

- 最低限のModelの作成しか行っていない。
- y_pred_gbでCategoryの予測が行われている。つまり、その事件が何であるかを予測しているという事、これは特徴量の性質を持つ事件が発生した場合、それが何であるかを予測して準備対応する備えが事前に可能という事でもある。事件が発生する確率も予測する事が可能。まるでトム・クルーズ主演のマイノリティリポート。

# XGBC

In [ ]:
xgb_model = XGBClassifier(use_label_encoder=True, metric='mlogloss')

In [ ]:
%timeit xgb_model.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb_model.predict(X_val)

In [ ]:
print('Precision : {} / Recall : {}'.format(precision_score(y_val, y_pred_xgb, average='micro'), recall_score(y_val, y_pred_xgb, average='micro')))
print(classification_report(y_val, y_pred_xgb))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_val, y_pred_xgb))
fig, ax = plt.subplots(1, 1, figsize=(19, 19))
disp.plot(ax=ax);

## GradientBoostingClassifier

In [ ]:
#gb_model = GradientBoostingClassifier()

In [ ]:
#gb_model.get_params().keys()

In [ ]:
#gb_param_grid = {'learning_rate':[0.1, 0.01, 0.001], 'max_depth':[5, 10], 'n_estimators':[10, 100, 200, 300]}

In [ ]:
#UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10. : stratify=data_labels
#gb_s_model = GridSearchCV(gb_model, gb_param_grid, cv=10, scoring='accuracy')
#gb_s_model.fit(X_train, y_train)

In [ ]:
# 訓練セットとテストセットの数が多すぎるので訓練終了に時間がかなりかかる。これを許容時間内に(テスト結果に関わらず)終わらせるには、シャッフル時に一定の訓練セットとテストセットを取り出す
gb_model = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=5)
%timeit gb_model.fit(X_train, y_train)

In [ ]:
y_pred_gb = gb_model.predict(X_val)

In [ ]:
print('Precision : {} / Recall : {}'.format(precision_score(y_val, y_pred_gb, average='micro'), recall_score(y_val, y_pred_gb, average='micro')))
print(classification_report(y_val, y_pred_gb))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_val, y_pred_gb))
fig, ax = plt.subplots(1, 1, figsize=(19, 19))
disp.plot(ax=ax);

--------
# Other Models Creating